### In this notebook we combine the Bayes classified data for French and German tweets and the given sentiments for English tweets and produce a dataframe, which contains all cities by months, sentiment and language.

In [1]:
import pandas as pd
import json
import numpy as np
import folium
import nltk
import time
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import math
from geopy.geocoders import Nominatim
import re
from mpl_toolkits.basemap import Basemap


In [3]:
bayes = pd.read_csv('processed/Bayes/bayesCombined.csv')
given = pd.read_csv('processed/sentiments/twitter_sentiments_combined.csv')
bayes.head()



,City,Sentiment,Count,Month,Language
0,source_location,sentiment,0,april,de
1,Aarau,NEGATIVE,263,april,de
2,Aarau,NEUTRAL,6,april,de
3,Aarau,POSITIVE,441,april,de
4,Affoltern am Albis,NEGATIVE,1,april,de


From the dataframe with given sentiments we only need the tweets in English.

In [4]:
given = given[given['Language']=='en']
given.head()

,City,Sentiment,Count,Month,Language
17685,Luzern,NEUTRAL,279,april,en
17686,Uster,POSITIVE,16,april,en
17687,Switzerland,POSITIVE,53815,april,en
17688,Winterthur,POSITIVE,152,april,en
17689,Porrentruy,NEUTRAL,2,april,en


Concatenating the two dataframes into one.

In [5]:
df = pd.concat([bayes,given])
df

,City,Sentiment,Count,Month,Language
0,source_location,sentiment,0,april,de
1,Aarau,NEGATIVE,263,april,de
2,Aarau,NEUTRAL,6,april,de
3,Aarau,POSITIVE,441,april,de
4,Affoltern am Albis,NEGATIVE,1,april,de
5,Affoltern am Albis,POSITIVE,3,april,de
6,Aigle,NEGATIVE,1,april,de
7,Altdorf,NEGATIVE,59,april,de
8,Altdorf,NEUTRAL,6,april,de
9,Altdorf,POSITIVE,62,april,de


List of words, where Switzerland is in different forms. We will remove records, which city field contains those, as we are interested in the locations such as cities, not countries.

In [6]:
switzerland = ['schweiz', 'schwyz', 'سويسرا', 'สวิตเซอร์แลนด์', 'スイス','thuy si','ch',
       'suica', 'suisse', 'suiza', 'suíça', 'suïssa', 'svajcarska',
       'sveitsi', 'svizzera', 'swiss', 'switzerland',
       'zwitserland', 'isviçre', 'Швейцария']

Cleaning up the city field. 

In [7]:
df['City'] = df['City'].map(lambda x: re.sub(r'[^áâçàéèa-zA-ZÖöÜüÄä]+', ' ', x)) #Only latin characters
df['City'] = df['City'].map(lambda x: x.split(' ', 1)[0]) #Selecting the first word
df['City'] = df['City'].map(lambda x: x.lower()) #Converting to lowercase
df = df.loc[~df['City'].isin(switzerland)] #Removing if conatins a word representing Switzerland
df['City'].replace('', np.nan, inplace=True) #Replacing just a whitespace with nan value for later removal

df = df[pd.notnull(df['City'])] #Keeping only records which have a city field
df

//anaconda/lib/python3.5/site-packages/pandas/core/generic.py:3443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,City,Sentiment,Count,Month,Language
0,source,sentiment,0,april,de
1,aarau,NEGATIVE,263,april,de
2,aarau,NEUTRAL,6,april,de
3,aarau,POSITIVE,441,april,de
4,affoltern,NEGATIVE,1,april,de
5,affoltern,POSITIVE,3,april,de
6,aigle,NEGATIVE,1,april,de
7,altdorf,NEGATIVE,59,april,de
8,altdorf,NEUTRAL,6,april,de
9,altdorf,POSITIVE,62,april,de


Keeping only the records for which we have positive or negative sentiments by discarding neutral. As there are many neutral sentiments, we believe it gives us a better idea about cities which tend to have positive or negative sentiments.

In [8]:
df = df.loc[df['Sentiment'].isin(['POSITIVE','NEGATIVE'])]
df

,City,Sentiment,Count,Month,Language
1,aarau,NEGATIVE,263,april,de
3,aarau,POSITIVE,441,april,de
4,affoltern,NEGATIVE,1,april,de
5,affoltern,POSITIVE,3,april,de
6,aigle,NEGATIVE,1,april,de
7,altdorf,NEGATIVE,59,april,de
9,altdorf,POSITIVE,62,april,de
10,amden,NEGATIVE,3,april,de
12,amden,POSITIVE,6,april,de
13,appenzell,NEGATIVE,7,april,de


In [9]:

#df['City'] = pd.core.strings.str_strip(df['City'])
#df.head()

Grouping the data by city, sentiment, month, language and adding up the counts for tweets.

In [10]:
df = df[['City', 'Sentiment', 'Count', 'Month','Language']]
df = df.groupby(['City','Sentiment','Month','Language'])[('Count')].sum()
df

City     Sentiment  Month      Language
a        NEGATIVE   august     de            1
                               en            1
                    july       de            1
                               fr            1
                    may        de            1
                               en            6
                               fr          200
                    september  en            2
                               fr            1
         POSITIVE   august     de            3
                               en            4
                               fr            2
                    july       de            1
                               en            2
                    may        en           20
                               fr           36
                    october    de            1
                               en            3
                               fr            1
                    september  de            1
                    

To recreate dataframe we reset index.

In [11]:
df = df.reset_index()
df

,City,Sentiment,Month,Language,Count
0,a,NEGATIVE,august,de,1
1,a,NEGATIVE,august,en,1
2,a,NEGATIVE,july,de,1
3,a,NEGATIVE,july,fr,1
4,a,NEGATIVE,may,de,1
5,a,NEGATIVE,may,en,6
6,a,NEGATIVE,may,fr,200
7,a,NEGATIVE,september,en,2
8,a,NEGATIVE,september,fr,1
9,a,POSITIVE,august,de,3


Selecting for each city, month, language the record, in which the count is the highest, so that the most frequent sentiment is left for each city.

In [13]:
df = df.loc[df.groupby(['City','Month','Language'])["Count"].idxmax()]
df

,City,Sentiment,Month,Language,Count
9,a,POSITIVE,august,de,3
10,a,POSITIVE,august,en,4
11,a,POSITIVE,august,fr,2
2,a,NEGATIVE,july,de,1
13,a,POSITIVE,july,en,2
3,a,NEGATIVE,july,fr,1
4,a,NEGATIVE,may,de,1
14,a,POSITIVE,may,en,20
6,a,NEGATIVE,may,fr,200
16,a,POSITIVE,october,de,1


As we already had the coordinates evaluated, we will reuse the file to add coordinates for the combined data.

In [14]:
bayesCoordinates = pd.read_csv('swissTwitterSentimentsAndCoordinates10mBayes.csv')
bayesCoordinates = bayesCoordinates[['City', 'Latitude', 'Longitude']]
bayesCoordinates.head()


,City,Latitude,Longitude
0,.ch,46.798562,8.231974
1,aadorf,47.491578,8.902953
2,aarau,47.392715,8.044445
3,aarau.,47.392715,8.044445
4,aarberg,47.044335,7.275300


In [15]:
givenCoordinates = pd.read_csv('swissTwitterSentimentsAndCoordinates10mGiven.csv')
givenCoordinates = givenCoordinates[['City', 'Latitude', 'Longitude']] 
givenCoordinates.head()

,City,Latitude,Longitude
0,.ch,46.798562,8.231974
1,aadorf,47.491578,8.902953
2,aarau,47.392715,8.044445
3,aarberg,47.044335,7.275300
4,aarburg,47.320642,7.899360


In [16]:
coordinates = pd.concat([bayesCoordinates,givenCoordinates])
coordinates.head()

,City,Latitude,Longitude
0,.ch,46.798562,8.231974
1,aadorf,47.491578,8.902953
2,aarau,47.392715,8.044445
3,aarau.,47.392715,8.044445
4,aarberg,47.044335,7.275300


In [17]:
coordinates.drop_duplicates()
coordinates


,City,Latitude,Longitude
0,.ch,46.798562,8.231974
1,aadorf,47.491578,8.902953
2,aarau,47.392715,8.044445
3,aarau.,47.392715,8.044445
4,aarberg,47.044335,7.275300
5,aarburg,47.320642,7.899360
6,aargau,47.412396,8.194832
7,adelboden,46.492721,7.558762
8,adetswil,47.339633,8.843533
9,adligenswil,47.070535,8.368244


Doing an inner join to assign coordinates for the combined data.

In [19]:
merged = pd.merge(df, coordinates, how='inner', on='City')

mergedClean = merged.drop_duplicates() #Dropping duplicates
mergedClean

,City,Sentiment,Month,Language,Count,Latitude,Longitude
0,aadorf,POSITIVE,august,de,56,47.491578,8.902953
2,aadorf,POSITIVE,august,en,4,47.491578,8.902953
4,aadorf,POSITIVE,july,de,5,47.491578,8.902953
6,aadorf,POSITIVE,october,de,57,47.491578,8.902953
8,aadorf,NEGATIVE,october,en,2,47.491578,8.902953
10,aadorf,POSITIVE,september,de,49,47.491578,8.902953
12,aadorf,POSITIVE,september,en,3,47.491578,8.902953
14,aarau,POSITIVE,april,de,441,47.392715,8.044445
16,aarau,POSITIVE,april,en,51,47.392715,8.044445
18,aarau,NEGATIVE,april,fr,3,47.392715,8.044445


In [20]:
mergedClean['City'] = mergedClean['City'].map(lambda x: x.capitalize())

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [21]:
mergedClean.head()

,City,Sentiment,Month,Language,Count,Latitude,Longitude
0,Aadorf,POSITIVE,august,de,56,47.491578,8.902953
2,Aadorf,POSITIVE,august,en,4,47.491578,8.902953
4,Aadorf,POSITIVE,july,de,5,47.491578,8.902953
6,Aadorf,POSITIVE,october,de,57,47.491578,8.902953
8,Aadorf,NEGATIVE,october,en,2,47.491578,8.902953


Writing the data into a file, from which it can be used to create the interactive map.

In [19]:
#outFile = "allDataWithMonthsCoordinatesLanguages.csv"
#mergedClean.to_csv(outFile,index=None)

In [23]:
a = pd.read_csv('allDataWithMonthsCoordinatesLanguages.csv')
a

,City,Sentiment,Month,Language,Count,Latitude,Longitude
0,Aadorf,POSITIVE,august,de,56,47.491578,8.902953
1,Aadorf,POSITIVE,august,en,4,47.491578,8.902953
2,Aadorf,NEGATIVE,july,de,5,47.491578,8.902953
3,Aadorf,POSITIVE,october,de,53,47.491578,8.902953
4,Aadorf,NEGATIVE,october,en,2,47.491578,8.902953
5,Aadorf,POSITIVE,september,de,51,47.491578,8.902953
6,Aadorf,POSITIVE,september,en,3,47.491578,8.902953
7,Aarau,POSITIVE,april,de,423,47.392715,8.044445
8,Aarau,POSITIVE,april,en,51,47.392715,8.044445
9,Aarau,POSITIVE,april,fr,4,47.392715,8.044445
